# Stack 10 Opt Flows + SVM

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import caffe
import os
from sklearn.svm import SVC
def load_list(data_dir, prefix = None, dataset = 1):
    train_list = os.path.join(data_dir, 'train') + str(dataset) + '.idx'
    test_list = os.path.join(data_dir, 'test') + str(dataset) + '.idx'
    with open(train_list) as f:
        train_list = f.read().splitlines()
    with open(test_list) as f:
        test_list = f.read().splitlines()
    if prefix is not None:
        train_list = map(lambda x: os.path.join(prefix, x), train_list)
        test_list = map(lambda x: os.path.join(prefix, x), test_list)
    return train_list, test_list

def load_img(filename, flip = False, grayscale = False):
    if grayscale:
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
        img = cv2.imread(filename)
    if flip:
        img = cv2.flip(img, 1)
    if grayscale:
        width, height = img.shape
    else:
        width, height, _ = img.shape
    factor = max(256.0 / width, 256.0 / height)
    img = cv2.resize(img, None, fx = factor, fy = factor)
    return img

def load_imgs(filenames, reverse = False, flip = False, grayscale = False):
    if reverse:
        filenames = filenames[::-1]
    imgs = map(lambda x: load_img(x, flip, grayscale), filenames)
    return imgs

def load_video(video, data_dir, mask = None, reverse = False, flip = False, grayscale = False):
    filenames = os.listdir(os.path.join(data_dir, video))
    filenames = filter(lambda x: x[-4:] == 'jpeg', filenames)
    if mask is not None:
        filenames = filter(mask, filenames)
    filenames.sort()
    filenames = map(lambda x: os.path.join(data_dir, video, x), filenames)
    imgs = load_imgs(filenames, reverse, flip, grayscale)
    return imgs

def is_forward(video):
    if video[0] == 'F':
        return True
    return False

def select(imgs):
    #mags = [None] * (len(imgs)/2)
    #for i in range(len(imgs)/2):
    #    mag, _ = cv2.cartToPolar(np.asarray(imgs[2*i], dtype = np.float32), np.asarray(imgs[2*i+1], dtype = np.float32))
    #mags[i] = cv2.norm(mag)
    #mags = np.asarray(mags)
    #idx = np.argsort(mags)[::-1][:10]
    idx = np.random.choice(len(imgs)/2, 10)
    indices = [None] * 20
    for i in range(10):
        indices[2*i] = 2*idx[i]
        indices[2*i+1] = 2*idx[i]+1
    image = np.take(np.asarray(imgs), indices, axis = 0)
    return image

def load_flows(video):
    images = []
    if is_forward(video):
        imgs = load_video(video, './data/ArrowDataAll', mask = lambda x: x[:3] == 'off', grayscale=True)
        imgs = map(lambda x: cv2.resize(x, (227, 227)), imgs)
        images.append(select(imgs))

        imgs = load_video(video, './data/ArrowDataAll', mask = lambda x: x[:3] == 'off', flip = True, grayscale=True)
        imgs = map(lambda x: cv2.resize(x, (227, 227)), imgs)
        images.append(select(imgs))

        imgs = load_video(video, './data/ArrowDataAll', mask = lambda x: x[:3] == 'ofb', grayscale=True)
        imgs = map(lambda x: cv2.resize(x, (227, 227)), imgs)
        images.append(select(imgs))
        
        imgs = load_video(video, './data/ArrowDataAll', mask = lambda x: x[:3] == 'ofb', flip = True, grayscale=True)
        imgs = map(lambda x: cv2.resize(x, (227, 227)), imgs)
        images.append(select(imgs))

        labels = [1, 1, 0, 0]
    else:
        imgs = load_video(video, './data/ArrowDataAll', mask = lambda x: x[:3] == 'ofb', grayscale=True)
        imgs = map(lambda x: cv2.resize(x, (227, 227)), imgs)
        images.append(select(imgs))
        
        imgs = load_video(video, './data/ArrowDataAll', mask = lambda x: x[:3] == 'ofb', flip = True, grayscale=True)
        imgs = map(lambda x: cv2.resize(x, (227, 227)), imgs)
        images.append(select(imgs))

        imgs = load_video(video, './data/ArrowDataAll', mask = lambda x: x[:3] == 'off', grayscale=True)
        imgs = map(lambda x: cv2.resize(x, (227, 227)), imgs)
        images.append(select(imgs))

        imgs = load_video(video, './data/ArrowDataAll', mask = lambda x: x[:3] == 'off', flip = True, grayscale=True)
        imgs = map(lambda x: cv2.resize(x, (227, 227)), imgs)
        images.append(select(imgs))

        labels = [0, 0, 1, 1]
    return np.asarray(images), np.asarray(labels)

In [2]:
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [3]:
caffe.set_mode_gpu()
# load original model
model_def = 'models/bvlc_reference_caffenet/deploy.prototxt'
model_weights = 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
net = caffe.Net(model_def, model_weights, caffe.TEST) 
# mean the conv1 layer across the 3 channels
conv1_mean = net.params['conv1'][0].data.mean(axis = 1)
# duplicate 20 times to handle 20-channel input data
conv1_stack = np.repeat(conv1_mean[:, np.newaxis, :, :], 20, axis = 1)

In [4]:
# load modified model 
# with layers later than fc7 removed
model_def = 'models/caffenet/deploy.prototxt'
model_weights = 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
net = caffe.Net(model_def, model_weights, caffe.TEST) 
# network surgery
net.params['conv1-stack'][0].data[...] = conv1_stack
net.blobs['data'].reshape(4, 20, 227, 227)

In [5]:
# load list 1
train_list, test_list = load_list('./data', dataset = 1)
X_train = np.empty((0, 4096))
y_train = np.empty(0)
for train in train_list:
    print '[INFO] processing video %d / %d' % (train_list.index(train) + 1, len(train_list))
    images, labels = load_flows(train)
    images -= 128*np.ones_like(images)
    net.blobs['data'].data[...] = images
    output = net.forward()
    X_train = np.append(X_train, output['fc7'], axis = 0)
    y_train = np.append(y_train, labels)

[INFO] processing video 1 / 120
[INFO] processing video 2 / 120
[INFO] processing video 3 / 120
[INFO] processing video 4 / 120
[INFO] processing video 5 / 120
[INFO] processing video 6 / 120
[INFO] processing video 7 / 120
[INFO] processing video 8 / 120
[INFO] processing video 9 / 120
[INFO] processing video 10 / 120
[INFO] processing video 11 / 120
[INFO] processing video 12 / 120
[INFO] processing video 13 / 120
[INFO] processing video 14 / 120
[INFO] processing video 15 / 120
[INFO] processing video 16 / 120
[INFO] processing video 17 / 120
[INFO] processing video 18 / 120
[INFO] processing video 19 / 120
[INFO] processing video 20 / 120
[INFO] processing video 21 / 120
[INFO] processing video 22 / 120
[INFO] processing video 23 / 120
[INFO] processing video 24 / 120
[INFO] processing video 25 / 120
[INFO] processing video 26 / 120
[INFO] processing video 27 / 120
[INFO] processing video 28 / 120
[INFO] processing video 29 / 120
[INFO] processing video 30 / 120
[INFO] processing v

In [6]:
X_test = np.empty((0, 4096))
y_test = np.empty(0)
for test in test_list:
    print '[INFO] processing video %d / %d' % (test_list.index(test) + 1, len(test_list))
    images, labels = load_flows(test)
    images -= 128*np.ones_like(images)
    net.blobs['data'].data[...] = images
    output = net.forward()
    X_test = np.append(X_test, output['fc7'], axis = 0)
    y_test = np.append(y_test, labels)

[INFO] processing video 1 / 60
[INFO] processing video 2 / 60
[INFO] processing video 3 / 60
[INFO] processing video 4 / 60
[INFO] processing video 5 / 60
[INFO] processing video 6 / 60
[INFO] processing video 7 / 60
[INFO] processing video 8 / 60
[INFO] processing video 9 / 60
[INFO] processing video 10 / 60
[INFO] processing video 11 / 60
[INFO] processing video 12 / 60
[INFO] processing video 13 / 60
[INFO] processing video 14 / 60
[INFO] processing video 15 / 60
[INFO] processing video 16 / 60
[INFO] processing video 17 / 60
[INFO] processing video 18 / 60
[INFO] processing video 19 / 60
[INFO] processing video 20 / 60
[INFO] processing video 21 / 60
[INFO] processing video 22 / 60
[INFO] processing video 23 / 60
[INFO] processing video 24 / 60
[INFO] processing video 25 / 60
[INFO] processing video 26 / 60
[INFO] processing video 27 / 60
[INFO] processing video 28 / 60
[INFO] processing video 29 / 60
[INFO] processing video 30 / 60
[INFO] processing video 31 / 60
[INFO] processing

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
lr = LogisticRegression(n_jobs=-1)

In [21]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
lr.score(X_train, y_train)

1.0

In [25]:
print 'raw precision %f' % lr.score(X_test, y_test)

raw precision 0.479167


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 1000)
pca.fit(X_train)
X_train_reduct = pca.transform(X_train)
X_test_reduct = pca.transform(X_test)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train_reduct, y_train)
print rfc.score(X_train_reduct, y_train)
print rfc.score(X_test_reduct, y_test)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train, y_train)

In [ ]:
rfc.score(X_train, y_train)

In [ ]:
rfc.score(X_test, y_test)

In [ ]:
plt.plot(rfc.feature_importances_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs = -1)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

In [ ]:
X_train[:4]

In [ ]:
y_predict_ = svc.predict(X_test)
y_predict = np.empty(len(y_predict_) / 4)
y_test = np.empty(len(y_predict))
for i in range(len(y_predict)):
    k = 4*i
    y_predict[i] = y_predict_[k] + y_predict_[k+1] - y_predict_[k+2] - y_predict_[k+3]
    y_test[i] = y_test_[k]
y_predict = np.sign(y_predict + 0.5)
precision = np.sum(y_predict == y_test) / float(len(y_test))
print 'Precision of dataset %d: %f' % (dataset, precision)
return precisionsvc.score(X_test, y_test)